# Task \#2: Encoding and Classifier

# Install and import dependencies

In [36]:
!pip install pennylane

In [37]:
import pennylane as qml
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import random

from itertools import chain
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

import pennylane as qml
from pennylane import numpy as np
from pennylane.templates.embeddings import AngleEmbedding
from pennylane.templates.layers import StronglyEntanglingLayers
from pennylane.optimize import GradientDescentOptimizer


# Import data

In [38]:
np.random.seed(42)

train = pd.read_csv('mock_train_set.csv')
test = pd.read_csv('mock_test_set.csv')

# Preprocess data

- Feature 0 is scaled between [-1,1];
- Feature 1 is encoded categorically;
- Feature 2 is encoded categorically;
- Feature 3 is scaled between [-1,1];

In [39]:
trainX = train[['0','1','2','3']].values
testX = test[['0','1','2','3']].values

scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(trainX[:,0].reshape(-1,1))
trainX[:,0] = scaler.transform(trainX[:,0].reshape(-1,1)).squeeze()
testX[:,0] = scaler.transform(testX[:,0].reshape(-1,1)).squeeze()

le = preprocessing.LabelEncoder()

le.fit(trainX[:,1].reshape(-1,1))
trainX[:,1] = le.transform(trainX[:,1].reshape(-1,1))
testX[:,1] = le.transform(testX[:,1].reshape(-1,1))


le2 = preprocessing.LabelEncoder()

le2.fit(trainX[:,2].reshape(-1,1))
trainX[:,2] = le2.transform(trainX[:,2].reshape(-1,1))
testX[:,2] = le2.transform(testX[:,2].reshape(-1,1))

scaler2 = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(trainX[:,3].reshape(-1,1))
trainX[:,3] = scaler.transform(trainX[:,3].reshape(-1,1)).squeeze()
testX[:,3] = scaler.transform(testX[:,3].reshape(-1,1)).squeeze()


trainX = Variable(torch.Tensor(trainX))
trainY = Variable(torch.Tensor(train['4'].values))


testX = Variable(torch.Tensor(testX))
testY = Variable(torch.Tensor(test['4'].values))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Angle Embedding

Encode data into qubits' angles through rotation gates and define three BasicEntanglerLayers: one with RZ, one with RX and one with RY rotations. All of them have CNOTs as entangling gates. **Two** repetitions of each entangling layer are defined.

Measure in Pauli-Z basis just the first qubit, and retrieve its expectation value.

In [40]:
n_qubits = 4
dev = qml.device("lightning.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch", diff_method='adjoint')
def qnode(inputs, weights):
    #print('inputs:',inputs,'\n\n','weights:',weights)
    w = weights.shape[0]//3
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights[:w], wires=range(n_qubits), rotation=qml.RZ)
    qml.BasicEntanglerLayers(weights[w:2*w], wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights[2*w:], wires=range(n_qubits), rotation=qml.RY)

    return [qml.expval(qml.PauliZ(wires=i)) for i in range(1)] #measure the first qubit only, as reported in...

n_layers=2
entangling_layers = 3
weight_shapes = {"weights": (n_layers*entangling_layers, n_qubits)}
print('weight shape:',weight_shapes)

qlayerAngleEmbedding = qml.qnn.TorchLayer(qnode, weight_shapes)

weight shape: {'weights': (6, 4)}


# Use Pytorch interface and define the Net class

In [41]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.qc = qlayerAngleEmbedding
        self.fctest = nn.Linear(4,1) #Useful for classical NN comparison
        #self.fc = nn.Linear(2,1)
        

    def forward(self, x):
        
        #print(x)
        x = self.qc(x)
        #x = self.fctest(x)
        
        return x
    
    
    def predict(self, x):
        # apply softmax
        pred = self.forward(x)
#         print(pred)
        #ans = torch.argmax(pred).item()
        return torch.tensor(pred)

# Training and testing

In [42]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score
import math
import torch.optim as optim

epochs = 20
torch.manual_seed(47)
random.seed(47)

network = Net()
    
optimizer = optim.Adam(network.parameters(), lr=0.001, weight_decay=1e-4)

train_loss_list = []
val_loss_list = []

network.train()

loss_func = nn.MSELoss()

for epoch in range(epochs):
  total_loss = []
  for batch_idx in range(len(trainY)):
    data, target = trainX[batch_idx], trainY[batch_idx].view(1)

    optimizer.zero_grad(set_to_none=True)        
    # Forward pass
    output = network(data)
    #print(output,target)
    # Calculating loss
    loss = loss_func(output, target)
    # Backward pass
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(network.parameters(), 1.0)
    # Optimize the weights
    optimizer.step()

    total_loss.append(loss.item())    
    print('\r Epoch %d ~ Loss %f ' % (epoch, loss.item()), end='\t\t')

  with torch.no_grad():
    val_loss = []
    targets = []
    predictions = []
    for batch_idx in range(len(testX)):
      data, target = testX[batch_idx], testY[batch_idx].view(1)
      output = network(data)
      loss = loss_func(output, target)

      val_loss.append(loss.item())

      targets.append(target)
      predictions.append(network.predict(data))

  train_loss_list.append(sum(total_loss)/len(total_loss))
  val_loss_list.append(sum(val_loss)/len(val_loss))

  print('Training [{:.0f}%]\t Training Loss: {:.4f} Test Loss: {:.4f}'.format(
      100. * (epoch + 1) / epochs, train_loss_list[-1], val_loss_list[-1]))

for i in range(len(predictions)):
  if predictions[i]>0.5:
    predictions[i]=1
  else:
    predictions[i]=0
    
print('\nAccuracy score on test: %.2f%%' % (accuracy_score(testY,predictions)*100))

 Epoch 0 ~ Loss 0.389044 		

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Training [5%]	 Training Loss: 0.6956 Test Loss: 0.5270
 Epoch 1 ~ Loss 0.310232 		Training [10%]	 Training Loss: 0.4559 Test Loss: 0.3583
 Epoch 2 ~ Loss 0.254295 		Training [15%]	 Training Loss: 0.3458 Test Loss: 0.2757
 Epoch 3 ~ Loss 0.209780 		Training [20%]	 Training Loss: 0.2902 Test Loss: 0.2301
 Epoch 4 ~ Loss 0.178556 		Training [25%]	 Training Loss: 0.2595 Test Loss: 0.2038
 Epoch 5 ~ Loss 0.158638 		Training [30%]	 Training Loss: 0.2417 Test Loss: 0.1881
 Epoch 6 ~ Loss 0.147348 		Training [35%]	 Training Loss: 0.2305 Test Loss: 0.1783
 Epoch 7 ~ Loss 0.142867 		Training [40%]	 Training Loss: 0.2225 Test Loss: 0.1718
 Epoch 8 ~ Loss 0.144002 		Training [45%]	 Training Loss: 0.2161 Test Loss: 0.1669
 Epoch 9 ~ Loss 0.149583 		Training [50%]	 Training Loss: 0.2102 Test Loss: 0.1630
 Epoch 10 ~ Loss 0.158246 		Training [55%]	 Training Loss: 0.2048 Test Loss: 0.1594
 Epoch 11 ~ Loss 0.168651 		Training [60%]	 Training Loss: 0.1996 Test Loss: 0.1560
 Epoch 12 ~ Loss 0.179923 		Tr

# Amplitude Embedding

Encode data into qubits' amplitude, so we can use 2 qubits only instead of 4 qubits (previous case). BasicEntanglerLayers: one with RZ, one with RX and one with RY rotations. All of them have CNOTs as entangling gates. **Three** repetitions of each entangling layer are defined.

Measure in Pauli-Z basis just the first qubit, and retrieve its expectation value.

In [43]:
n_qubits = 2
dev = qml.device("lightning.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch", diff_method='adjoint')
def qnode(inputs, weights):
    #print('inputs:',inputs,'\n\n','weights:',weights)
    w = weights.shape[0]//3
    qml.AmplitudeEmbedding(inputs, wires=range(n_qubits), normalize=True)
    qml.BasicEntanglerLayers(weights[:w], wires=range(n_qubits), rotation=qml.RZ)
    qml.BasicEntanglerLayers(weights[w:2*w], wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights[2*w:], wires=range(n_qubits), rotation=qml.RY)
    #qml.AngleEmbedding(inputs, wires=range(n_qubits))
    #qml.BasicEntanglerLayers(weights[8:], wires=range(n_qubits))

    return [qml.expval(qml.PauliZ(wires=i)) for i in range(1)] #measure the first qubit only, as reported in...

n_layers=3
entangling_layers = 3
weight_shapes = {"weights": (n_layers*entangling_layers, n_qubits)}
print('weight shape:',weight_shapes)

qlayerAmplitudeEmbedding = qml.qnn.TorchLayer(qnode, weight_shapes)

weight shape: {'weights': (9, 2)}


# Use Pytorch interface and define the Net2 class

In [44]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        
        self.qc = qlayerAmplitudeEmbedding
        self.fctest = nn.Linear(4,1) #Useful for classical NN comparison
        #self.fc = nn.Linear(2,1)
        

    def forward(self, x):
        
        #print(x)
        x = self.qc(x)
        #x = self.fctest(x)
        
        return x
    
    
    def predict(self, x):
        # apply softmax
        pred = self.forward(x)
#         print(pred)
        #ans = torch.argmax(pred).item()
        return torch.tensor(pred)

# Training and testing

In [45]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score
import math
import torch.optim as optim

epochs = 20
torch.manual_seed(47)
random.seed(47)

network = Net2()
    
optimizer = optim.Adam(network.parameters(), lr=0.001, weight_decay=1e-4)

train_loss_list = []
val_loss_list = []

network.train()

loss_func = nn.MSELoss()

for epoch in range(epochs):
  total_loss = []
  for batch_idx in range(len(trainY)):
    data, target = trainX[batch_idx], trainY[batch_idx].view(1)

    optimizer.zero_grad(set_to_none=True)        
    # Forward pass
    output = network(data)
    #print(output,target)
    # Calculating loss
    loss = loss_func(output, target)
    # Backward pass
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(network.parameters(), 1.0)
    # Optimize the weights
    optimizer.step()

    total_loss.append(loss.item())    
    print('\r Epoch %d ~ Loss %f ' % (epoch, loss.item()), end='\t\t')

  with torch.no_grad():
    val_loss = []
    targets = []
    predictions = []
    for batch_idx in range(len(testX)):
      data, target = testX[batch_idx], testY[batch_idx].view(1)
      output = network(data)
      loss = loss_func(output, target)

      val_loss.append(loss.item())

      targets.append(target)
      predictions.append(network.predict(data))

  train_loss_list.append(sum(total_loss)/len(total_loss))
  val_loss_list.append(sum(val_loss)/len(val_loss))

  print('Training [{:.0f}%]\t Training Loss: {:.4f} Test Loss: {:.4f}'.format(
      100. * (epoch + 1) / epochs, train_loss_list[-1], val_loss_list[-1]))

for i in range(len(predictions)):
  if predictions[i]>0.5:
    predictions[i]=1
  else:
    predictions[i]=0
    
print('\nAccuracy score on test: %.2f%%' % (accuracy_score(testY,predictions)*100))

 Epoch 0 ~ Loss 0.753418 		

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Training [5%]	 Training Loss: 0.4044 Test Loss: 0.2959
 Epoch 1 ~ Loss 0.326499 		Training [10%]	 Training Loss: 0.2222 Test Loss: 0.1635
 Epoch 2 ~ Loss 0.187862 		Training [15%]	 Training Loss: 0.1668 Test Loss: 0.1192
 Epoch 3 ~ Loss 0.133999 		Training [20%]	 Training Loss: 0.1500 Test Loss: 0.1034
 Epoch 4 ~ Loss 0.108292 		Training [25%]	 Training Loss: 0.1441 Test Loss: 0.0962
 Epoch 5 ~ Loss 0.094223 		Training [30%]	 Training Loss: 0.1413 Test Loss: 0.0923
 Epoch 6 ~ Loss 0.085576 		Training [35%]	 Training Loss: 0.1396 Test Loss: 0.0898
 Epoch 7 ~ Loss 0.079653 		Training [40%]	 Training Loss: 0.1384 Test Loss: 0.0881
 Epoch 8 ~ Loss 0.075217 		Training [45%]	 Training Loss: 0.1375 Test Loss: 0.0869
 Epoch 9 ~ Loss 0.071675 		Training [50%]	 Training Loss: 0.1368 Test Loss: 0.0859
 Epoch 10 ~ Loss 0.068727 		Training [55%]	 Training Loss: 0.1362 Test Loss: 0.0851
 Epoch 11 ~ Loss 0.066207 		Training [60%]	 Training Loss: 0.1357 Test Loss: 0.0845
 Epoch 12 ~ Loss 0.064016 		Tr

# Discussion

Angle embedding allows to achieve slightly better performances with respect to amplitude encoding, although it uses as many qubits as the # of features. Amplitude encoding, instead, just uses $log_2(d)$ qubits, where $d$ is the number of features.

In both the configurations, rotations around X, Y and Z axes are necessary to increase the expressivity of the circuit and to better approximate the underlying function.